In [73]:
%matplotlib inline
import numpy as np
import re
from matplotlib import pyplot as plt 

import sys 
sys.setrecursionlimit(4000000)

import pickle
import h5py



## Read in reads data for chromosome from h5py folder

In [74]:
# TEST WITH CHR4

wantedchrom = 'chr4'

# get reads
readfile = h5py.File('h5py_compressed/reads_sampledata_' + wantedchrom + '.h5','r')
reads = readfile.get('reads')

# get annotations & chromosome length
annofile = h5py.File('h5py_compressed/dm6_annotations/' + wantedchrom + '.h5','r')
chromslens = h5py.File('h5py_compressed/CHROMS_LEN.h5', 'r')
length = chromslens.get(wantedchrom)[()]

## Gets list of all above-threshold indices for the given chromosome 

In [75]:
# PEAKSLIST = [(#reads, index, peak#)...]
THRESHOLD = 10

peakslist = []
counter = 0
peaknum = 0

for idx in range(length):    
    if reads[idx] > THRESHOLD:
        if len(peakslist) > 0:
            if idx != peakslist[-1][1] + 1:
                peaknum += 1
        peakslist.append((reads[idx],idx, peaknum)) 
    counter += 1
    if counter % 10000 == 0:
        print str(counter) + ' of ' + str(length)

10000 of 1348131
20000 of 1348131
30000 of 1348131
40000 of 1348131
50000 of 1348131
60000 of 1348131
70000 of 1348131
80000 of 1348131
90000 of 1348131
100000 of 1348131
110000 of 1348131
120000 of 1348131
130000 of 1348131
140000 of 1348131
150000 of 1348131
160000 of 1348131
170000 of 1348131
180000 of 1348131
190000 of 1348131
200000 of 1348131
210000 of 1348131
220000 of 1348131
230000 of 1348131
240000 of 1348131
250000 of 1348131
260000 of 1348131
270000 of 1348131
280000 of 1348131
290000 of 1348131
300000 of 1348131
310000 of 1348131
320000 of 1348131
330000 of 1348131
340000 of 1348131
350000 of 1348131
360000 of 1348131
370000 of 1348131
380000 of 1348131
390000 of 1348131
400000 of 1348131
410000 of 1348131
420000 of 1348131
430000 of 1348131
440000 of 1348131
450000 of 1348131
460000 of 1348131
470000 of 1348131
480000 of 1348131
490000 of 1348131
500000 of 1348131
510000 of 1348131
520000 of 1348131
530000 of 1348131
540000 of 1348131
550000 of 1348131
560000 of 1348131
5

In [101]:
peakslist[1000:2000]

[(18.0, 7498, 10),
 (17.0, 7499, 10),
 (18.0, 7500, 10),
 (18.0, 7501, 10),
 (19.0, 7502, 10),
 (19.0, 7503, 10),
 (19.0, 7504, 10),
 (18.0, 7505, 10),
 (17.0, 7506, 10),
 (18.0, 7507, 10),
 (17.0, 7508, 10),
 (15.0, 7509, 10),
 (15.0, 7510, 10),
 (15.0, 7511, 10),
 (16.0, 7512, 10),
 (16.0, 7513, 10),
 (15.0, 7514, 10),
 (16.0, 7515, 10),
 (16.0, 7516, 10),
 (15.0, 7517, 10),
 (15.0, 7518, 10),
 (15.0, 7519, 10),
 (15.0, 7520, 10),
 (14.0, 7521, 10),
 (15.0, 7522, 10),
 (14.0, 7523, 10),
 (14.0, 7524, 10),
 (14.0, 7525, 10),
 (14.0, 7526, 10),
 (14.0, 7527, 10),
 (14.0, 7528, 10),
 (13.0, 7529, 10),
 (13.0, 7530, 10),
 (13.0, 7531, 10),
 (13.0, 7532, 10),
 (13.0, 7533, 10),
 (13.0, 7534, 10),
 (13.0, 7535, 10),
 (12.0, 7536, 10),
 (11.0, 7537, 10),
 (11.0, 7538, 10),
 (11.0, 7539, 10),
 (11.0, 7540, 10),
 (11.0, 7541, 10),
 (11.0, 7542, 10),
 (11.0, 7543, 10),
 (11.0, 7544, 10),
 (13.0, 7545, 10),
 (13.0, 7546, 10),
 (13.0, 7547, 10),
 (13.0, 7548, 10),
 (12.0, 7549, 10),
 (12.0, 7550

## SIMPLE PEAK DATA --> FILE?

In [99]:
# PEAKSLIST --> SIMPLE_PEAKSLIST: [(start, end, argmax, max)...]

# NOTE: THIS STORES THE ARGMAX AS THE FIRST APPEARANCE OF THE MAX, 
# SO IF THERE ARE MULTIPLE OCCURRENCES OF THE MAX, IT IS VERY MUCH NOT CENTERED

previndex = -1
numpeaks = peakslist[-1][2] + 1
maxes = np.zeros(numpeaks, dtype=int)
argmaxes = np.zeros(numpeaks, dtype=int)
starts = np.zeros(numpeaks, dtype=int); starts.fill(-1)
ends = np.zeros(numpeaks, dtype=int); ends.fill(-1)
peak_id = 0

for triple in peakslist:
    numreads = int(triple[0])
    idx = int(triple[1])
    peaknum = int(triple[2])
    
    if numreads > maxes[peaknum]:
        maxes[peaknum] = numreads
        argmaxes[peaknum] = idx
    if (idx < starts[peaknum]) or (starts[peaknum] < 0):
        starts[peaknum] = idx
    ends[peaknum] = max(idx, ends[peaknum])
    
        
simple_peaks = []
for i in range(numpeaks):
    simple_peaks.append((starts[i], ends[i], argmaxes[i], maxes[i]))



In [100]:
simple_peaks

[(5886, 5893, 5890, 12),
 (5905, 5916, 5905, 11),
 (5918, 5930, 5919, 12),
 (5994, 6225, 6178, 52),
 (6381, 6537, 6470, 67),
 (6540, 6758, 6629, 67),
 (6941, 7020, 6972, 17),
 (7071, 7077, 7071, 11),
 (7088, 7088, 7088, 11),
 (7093, 7147, 7103, 15),
 (7282, 7561, 7457, 26),
 (7563, 7566, 7563, 11),
 (7568, 7604, 7571, 13),
 (7750, 7987, 7935, 124),
 (7991, 7992, 7992, 12),
 (7994, 8322, 8050, 117),
 (8347, 8687, 8568, 155),
 (8741, 8984, 8797, 118),
 (8988, 9084, 9034, 64),
 (9098, 9387, 9153, 100),
 (9400, 9754, 9653, 121),
 (9772, 10731, 10650, 295),
 (10733, 11049, 10791, 227),
 (11061, 11541, 11202, 105),
 (11555, 11752, 11609, 82),
 (12667, 12674, 12671, 12),
 (12686, 12697, 12686, 11),
 (12699, 12711, 12700, 12),
 (12775, 12877, 12834, 19),
 (12924, 13006, 12956, 47),
 (13032, 13126, 13073, 53),
 (13138, 13319, 13202, 72),
 (13322, 13544, 13411, 67),
 (14625, 14709, 14661, 19),
 (14798, 15090, 14976, 32),
 (15102, 15109, 15102, 11),
 (15150, 15171, 15155, 12),
 (15223, 15349, 152

In [98]:
max(5,6)

6